In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import date, timedelta



In [ ]:
from vnstock import Listing, Quote
list = Listing(source='vci')
VN30 = list.symbols_by_group('VN30').tolist()
#tickers = [t for t in VN30]
day_range = 180
tickers = ['HCM', 'ACB', 'HPG', 'SSI']
end = (date.today()).isoformat()
start = (date.today() - timedelta(days=day_range)).isoformat()



In [7]:
import fin_helpers.fetching.fetch_info as fetch

raw = fetch.fetch_stock(tickers=tickers,interval='1d',hist=day_range, combine=False)


Done!

Processing batch ['HPG' 'SSI']
Done!



In [8]:
for i in raw:
    stock = raw[i]
    stock['ret'] = np.log(stock['close'] / stock['close'].shift(1)) 
    stock.dropna(subset=['ret'], inplace=True)
    stock['ticker'] = i
    print(stock.isna().sum().sum())


0
0
0
0


In [9]:
df = []
for i in raw:
    df.append(raw[i])
df = pd.concat(df)
df['mvol'] = df['close'] * df['volume']
df = df[['time','ticker','close', 'mvol', 'ret']]

In [11]:
from IPython.display import clear_output   
import fin_helpers.finana.mpt as mpt

sims = 4000

result = mpt.mpt_sim(df, sims)

result

Index(['ACB', 'HCM', 'HPG', 'SSI'], dtype='object', name='ticker')


,Return,Volatility,Weight
3334,0.001976,0.024933,"{'ACB': 0.04871883338479502, 'HCM': 0.01841812..."
875,0.001962,0.024973,"{'ACB': 0.0450147840891257, 'HCM': 0.027745464..."
2125,0.001949,0.024202,"{'ACB': 0.08761901674635784, 'HCM': 0.00693924..."
2248,0.001939,0.024087,"{'ACB': 0.027059749854827522, 'HCM': 0.0001168..."
2354,0.001938,0.024337,"{'ACB': 0.07328483428361661, 'HCM': 0.01734801..."
...,...,...,...
463,-0.000202,0.021010,"{'ACB': 0.0036136677217734536, 'HCM': 0.899216..."
3079,-0.000206,0.020910,"{'ACB': 0.05309846003606885, 'HCM': 0.90008754..."
2220,-0.000222,0.020705,"{'ACB': 0.04326546841994761, 'HCM': 0.89401754..."
568,-0.000247,0.020989,"{'ACB': 0.00716350790887391, 'HCM': 0.91574479..."


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
aggregate = df.groupby('ticker')['ret'].agg(['mean', 'std','var'])  

plt.figure(figsize=(5,5))
plt.scatter(aggregate['var'], aggregate['mean'])

for x, y, label in zip(aggregate['var'], aggregate['mean'], aggregate.index):
    plt.annotate(label, (x, y), xytext=(4, 4), textcoords="offset points", fontsize=8)



In [ ]:
W = pd.DataFrame(result["Weight"].to_list(), index=result.index).add_prefix("w_")
result2 = pd.concat([result.drop(columns=["Weight"]), W], axis=1)
fin = result2.sort_values(['Return', 'Volatility'], ascending=[False, True])
fin


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import math 

fig, axs = plt.subplots(1, len(tickers), figsize=(20, 5), sharex=False, sharey=False)


for i in range(len(tickers)):
    sns.scatterplot(x=fin['Volatility'] * math.sqrt(day_range), y=fin['Return'] * day_range,hue=fin[f'w_{tickers[i]}'], ax=axs[i])


In [ ]:
fin.head(100).agg(['mean', 'std'])

In [ ]:
sns.lineplot(x=df['time'], y=df['close'], hue=df['ticker'])